In [1]:
import os
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
import sys
from pathlib import Path

PROJECT_ROOT = Path("/data/ephemeral/home/pro-cv-finalproject-cv-07/python") 
sys.path.insert(0, str(PROJECT_ROOT))

from src.configs.train_config import TrainConfig
from src.utils.set_seed import set_seed
from src.data.dataset import build_multi_item_dataset, deepar_split, lag_features_by_1day
from collections import defaultdict
import tyro
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json

from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from pytorch_lightning.loggers import CSVLogger

/data/ephemeral/home/py310/lib/python3.10/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:





print("\n" + "="*60)
print("📁 Loading Dataset")
print("="*60)



df={}
for name in ["corn", "wheat", "soybean", "gold", "silver", "copper"]:
    data_path = os.path.join("/data/ephemeral/home/pro-cv-finalproject-cv-07/python/src/datasets", f"preprocessing/{name}_feature_engineering.csv")
    data = pd.read_csv(data_path)
    data["item_id"] = name
    data['time'] = pd.to_datetime(data['time'])
    
    df[name] = data
  
    
    print(f" {name}: {len(data)} samples")




📁 Loading Dataset
 corn: 3824 samples
 wheat: 2738 samples
 soybean: 2670 samples
 gold: 3643 samples
 silver: 3643 samples
 copper: 3643 samples


In [3]:
df['corn']

,time,open,high,low,close,EMA,Volume,log_return_1,log_return_5,log_return_10,...,timeframe_score_std,timeframe_score_max,timeframe_score_min,sentiment_neg_ratio,sentiment_neu_ratio,sentiment_pos_ratio,time_past_ratio,time_present_ratio,time_future_ratio,item_id
0,2010-09-22,504.50,512.00,500.25,505.00,392.281730,187576,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
1,2010-09-23,506.25,507.00,495.25,499.25,393.346091,192998,-0.011451,0.000000,-0.033219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
2,2010-09-24,498.75,523.50,496.50,521.75,394.623742,201964,0.044082,-0.007035,-0.002005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
3,2010-09-27,522.00,528.75,511.25,512.75,395.799127,176759,-0.017400,-0.113540,0.012381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
4,2010-09-28,512.00,513.50,499.50,500.00,396.835952,151856,-0.025180,-0.083869,0.080530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819,2025-11-20,429.75,431.25,425.75,426.50,428.042614,194578,-0.007591,-0.013470,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
3820,2025-11-21,426.25,427.75,424.00,425.50,428.017315,194578,-0.002347,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
3821,2025-11-24,426.00,426.50,422.25,423.75,427.974854,175584,-0.004121,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn
3822,2025-11-25,423.75,426.00,422.25,423.50,427.930328,203255,-0.000590,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,corn


In [25]:
for i in range(len(arr)):
    if arr[i]>100:
        print(i)

23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [1]:
import inspect
from gluonts.torch.model.deepar import DeepAREstimator
print(inspect.signature(DeepAREstimator))

Disabling PyTorch because PyTorch >= 2.2 is required but found 2.1.0
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


(freq: str, prediction_length: int, context_length: Optional[int] = None, num_layers: int = 2, hidden_size: int = 40, lr: float = 0.001, weight_decay: float = 1e-08, dropout_rate: float = 0.1, patience: int = 10, num_feat_dynamic_real: int = 0, num_feat_static_cat: int = 0, num_feat_static_real: int = 0, cardinality: Optional[List[int]] = None, embedding_dimension: Optional[List[int]] = None, distr_output: gluonts.torch.distributions.distribution_output.DistributionOutput = gluonts.torch.distributions.studentT.StudentTOutput(beta=0.0), scaling: bool = True, default_scale: Optional[float] = None, lags_seq: Optional[List[int]] = None, time_features: Optional[List[Callable[[pandas.core.indexes.period.PeriodIndex], numpy.ndarray]]] = None, num_parallel_samples: int = 100, batch_size: int = 32, num_batches_per_epoch: int = 50, imputation_method: Optional[gluonts.transform.feature.MissingValueImputation] = None, trainer_kwargs: Optional[Dict[str, Any]] = None, train_sampler: Optional[gluonts

/data/ephemeral/home/py310/lib/python3.10/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [21]:
for a,i in df['soybean'].items():
    print(a)
    arr.append(sum(i.isna()))

time
open
high
low
close
EMA
Volume
log_return_1
log_return_5
log_return_10
log_return_20
EMA_5
EMA_10
EMA_20
EMA_50
EMA_100
vol_return_7d
vol_volume_7d
vol_return_14d
vol_volume_14d
vol_return_21d
vol_volume_21d
news_count
sentiment_score_mean
sentiment_score_std
sentiment_score_max
sentiment_score_min
timeframe_score_mean
timeframe_score_std
timeframe_score_max
timeframe_score_min
sentiment_neg_ratio
sentiment_neu_ratio
sentiment_pos_ratio
time_past_ratio
time_present_ratio
time_future_ratio
item_id


In [8]:
sum(df['soybean']['time_future_ratio'].isna())

653

In [4]:
feature_cols = [
        c for c in pd.concat(df.values(), ignore_index=True).columns
        if c not in ["time", "item_id", "close"] and not c.startswith("log_return_")
    ]

In [5]:
for name in list(df.keys()):
    df[name] = lag_features_by_1day(df[name], feature_cols, group_col="item_id", time_col="time")
    # 여기서 1day 땡길때 없애면 안되잖아 없애자 시원하게


In [1]:
df['corn']

NameError: name 'df' is not defined

In [5]:
for item_id, dfs in df.items():          
    dfs = dfs.sort_values("time").reset_index(drop=True)

    dfs["time_idx"] = np.arange(len(dfs), dtype=np.int64)

    df[item_id] = dfs 

In [6]:
df

{'corn':            time    open    high     low   close         EMA    Volume  \
 0    2017-11-10  347.50  349.75  340.75  343.50  359.237260  475087.0   
 1    2017-11-13  341.75  344.00  341.00  342.25  359.080670  349164.0   
 2    2017-11-14  343.50  344.00  341.50  337.50  358.913201  289722.0   
 3    2017-11-15  342.00  342.25  337.25  338.25  358.700134  220983.0   
 4    2017-11-16  337.25  339.00  337.00  336.50  358.496650  217010.0   
 ...         ...     ...     ...     ...     ...         ...       ...   
 2012 2025-11-11  426.75  431.25  426.25  432.00  427.605082  220392.0   
 2013 2025-11-12  429.00  432.75  428.50  435.25  427.648812  221848.0   
 2014 2025-11-13  431.50  436.00  431.00  441.50  427.724446  271699.0   
 2015 2025-11-14  434.75  442.75  434.00  430.25  427.861516  400924.0   
 2016 2025-11-17  441.50  442.75  429.50  434.75  427.885282  388692.0   
 
       log_return_1  log_return_5  log_return_10  ...  timeframe_score_max  \
 0         0.005839     

In [7]:
import inspect
print(DeepAREstimator)
print(inspect.getmodule(DeepAREstimator))


<class 'gluonts.torch.model.deepar.estimator.DeepAREstimator'>
<module 'gluonts.torch.model.deepar.estimator' from '/data/ephemeral/home/py310/lib/python3.10/site-packages/gluonts/torch/model/deepar/estimator.py'>


In [1]:
df['corn']

NameError: name 'df' is not defined

In [10]:
folds = [7] 


all_results_log = []
all_results_close = []

for fold in folds:
    print(f"\n{'='*60}")
    print(f"🔄 Processing Fold {fold}")
    print(f"{'='*60}")
    
    train_dfs = {}
    val_dfs = {}
    
    for name, dfs in df.items():
        train_df, val_df = deepar_split(
            dfs,
            os.path.join("/data/ephemeral/home/pro-cv-finalproject-cv-07/python/src/datasets", "rolling_fold.json"),
            fold,
        )
        train_dfs[name] = train_df
        val_dfs[name] = val_df
        train_ds = build_multi_item_dataset(train_dfs,"log_return_1",feature_cols)
        val_ds   = build_multi_item_dataset(val_dfs,"log_return_1",feature_cols)
        estimator = DeepAREstimator(
                freq="D",
                prediction_length=5,
                context_length=20,
                num_feat_dynamic_real=len(feature_cols),
                num_layers=3,
                hidden_size=64,
                dropout_rate=0.1,
                lr=1e-4,
                trainer_kwargs={
                    "max_epochs": 1,
                    "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
                    "devices": 1,
                    "gradient_clip_val": 1.0,
                },
            )
            
        predictor = estimator.train(
            training_data=train_ds,
            validation_data=val_ds
        )

        forecast_it, ts_it = make_evaluation_predictions(
            dataset=val_ds,
            predictor=predictor,
            num_samples=200,
        )
        forecasts = list(forecast_it)
        tss = list(ts_it)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type        | Params | Mode  | FLOPs | In sizes                                                          | Out sizes  
----------------------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 101 K  | train | 161 K | [[1, 1], [1, 1], [1, 1112, 35], [1, 1112], [1, 1112], [1, 5, 35]] | [1, 100, 5]
----------------------------------------------------------------------------------------------------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.404     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
161 K     Total Flops



🔄 Processing Fold 7
Item: corn, rows=765, n_feat=31
Item: corn, rows=126, n_feat=31


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Item: corn, rows=765, n_feat=31
Item: wheat, rows=765, n_feat=31
Item: soybean, rows=765, n_feat=31
Item: gold, rows=765, n_feat=31
Item: silver, rows=765, n_feat=31
Item: copper, rows=765, n_feat=31
Item: corn, rows=126, n_feat=31
Item: wheat, rows=126, n_feat=31
Item: soybean, rows=126, n_feat=31
Item: gold, rows=126, n_feat=31
Item: silver, rows=126, n_feat=31
Item: copper, rows=126, n_feat=31


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'val_loss' reached -2.90106 (best -2.90106), saving model to '/data/ephemeral/home/pro-cv-finalproject-cv-07/python/scripts/lightning_logs/version_19/checkpoints/epoch=0-step=50.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=1` reached.


TypeError: 'PandasDataset' object is not subscriptable

In [11]:
first = list(val_ds)[0]
print(type(first), first.keys())

<class 'dict'> dict_keys(['start', 'target', 'item_id', 'feat_dynamic_real'])


In [16]:
val_df

,time,open,high,low,close,EMA,Volume,log_return_1,log_return_5,log_return_10,...,timeframe_score_max,timeframe_score_min,sentiment_neg_ratio,sentiment_neu_ratio,sentiment_pos_ratio,time_past_ratio,time_present_ratio,time_future_ratio,item_id,time_idx
0,2024-05-29,4.7600,4.9030,4.7510,4.7910,4.082686,101463.0,-0.013270,-0.067743,-0.073712,...,0.999774,0.496353,0.410256,0.410256,0.179487,0.205128,0.230769,0.564103,copper,1646
1,2024-05-30,4.8750,4.8935,4.7500,4.6585,4.089734,71896.0,-0.028046,-0.039379,-0.047883,...,0.999788,0.927121,0.193548,0.483871,0.322581,0.225806,0.161290,0.612903,copper,1647
2,2024-05-31,4.7810,4.7840,4.6205,4.6020,4.095393,84255.0,-0.012203,0.004177,-0.038736,...,0.999786,0.613561,0.270833,0.500000,0.229167,0.041667,0.145833,0.812500,copper,1648
3,2024-06-03,4.6610,4.6735,4.5710,4.6675,4.100434,74629.0,0.014133,-0.026087,-0.023525,...,0.999794,0.575466,0.102041,0.408163,0.489796,0.183673,0.163265,0.653061,copper,1649
4,2024-06-04,4.6250,4.6920,4.5955,4.5370,4.106076,65807.0,-0.028358,-0.026926,-0.048281,...,0.999763,0.723079,0.277778,0.500000,0.222222,0.083333,0.222222,0.694444,copper,1650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2024-11-19,4.0665,4.1355,4.0505,4.1420,4.250191,50803.0,0.005326,-0.004866,0.019707,...,0.999722,0.544919,0.272727,0.500000,0.227273,0.090909,0.181818,0.727273,copper,1767
122,2024-11-20,4.1240,4.1745,4.0885,4.1555,4.249115,50275.0,0.003254,-0.022709,0.014263,...,0.999763,0.781507,0.062500,0.437500,0.500000,0.062500,0.250000,0.687500,copper,1768
123,2024-11-21,4.1660,4.1790,4.1365,4.1250,4.248183,44315.0,-0.007367,-0.003979,0.008864,...,0.999761,0.638620,0.071429,0.785714,0.142857,0.071429,0.214286,0.714286,copper,1769
124,2024-11-22,4.1530,4.1665,4.0970,4.0860,4.246958,45938.0,-0.009500,0.003630,0.017185,...,0.999776,0.868944,0.241379,0.517241,0.241379,0.103448,0.000000,0.896552,copper,1770


In [ ]:
tem_id = getattr(forecast_it, "item_id", None)
print(tem_id)

None
